# Simple autoencoder

In [1]:
import numpy as np
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def load_data(img_dir, ending):
    return np.array([cv2.imread(os.path.join(img_dir, img)) for img in os.listdir(img_dir) if img.endswith(ending)])


In [3]:
X_ORG = load_data("3Shapes2_large/", "im1.png")
X_ORG.shape

(8015, 128, 128, 3)

In [4]:
X = X_ORG[:1]

In [24]:
def create_graph():
    input_img = tf.placeholder(dtype=tf.float32, shape=[None,128,128,3], name="input")
    
    logits = tf.contrib.layers.flatten(input_img)
    
    w = tf.Variable(tf.random_normal([(128*128*3),(64*64*3)]))
    b = tf.Variable(tf.zeros((64*64*3)))
    logits = tf.add(tf.matmul(logits,w),b)
    logits = tf.nn.relu(logits)
    
    w = tf.Variable(tf.random_normal([(64*64*3),(16*16*3)]))
    b = tf.Variable(tf.zeros((16*16*3)))
    logits = tf.add(tf.matmul(logits,w),b)
    logits = tf.nn.relu(logits)
    
    w = tf.Variable(tf.random_normal([(16*16*3),(8*8*3)]))
    b = tf.Variable(tf.zeros((8*8*3)))
    logits = tf.add(tf.matmul(logits,w),b)
    logits = tf.nn.relu(logits)
                                      
    #uncompressing
                                      
    w = tf.Variable(tf.random_normal([(8*8*3),(16*16*3)]))
    b = tf.Variable(tf.zeros((16*16*3)))
    logits = tf.add(tf.matmul(logits,w),b)
    logits = tf.nn.relu(logits)
                                      
    w = tf.Variable(tf.random_normal([(16*16*3),(128*128*3)]))
    b = tf.Variable(tf.zeros((128*128*3)))
    logits = tf.add(tf.matmul(logits,w),b)
    
    logits = tf.reshape(logits, [-1, 128,128,3])
    
    return input_img, logits

In [25]:
def train_fn(x,y):
    loss = tf.reduce_mean(tf.square(x-y))
    optimizer = tf.train.AdamOptimizer(0.01)
    train = optimizer.minimize(loss)
    
    return loss, train

In [26]:
def run(x_train, epochs, num_batches):
    
    org_input, output = create_graph()
    loss, train = train_fn(org_input, output)
    
    with tf.Session() as sess:    
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            print("\nEpoch %i/%i:" % (epoch+1, epochs))
            t
            _, batch_loss = sess.run([train, loss], feed_dict={org_input:x_train})
            
            print("\tloss:", batch_loss)

In [ ]:
run(X,10,1)